In [1]:
import numpy as np
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [2]:
from keras import backend as K
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Conv2D, Flatten, MaxPooling2D, Reshape, Activation, Embedding, TimeDistributed
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
import pandas as pd

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

/home/zhouz2/.conda/envs/rbenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df = pd.read_csv('reuters.csv', delimiter='\t', names=['id', 'timestamp', 'title', 'url', 'first_line'])

In [4]:
df = df.set_index('id')

In [5]:
batch_size = 64  # Batch size for training.
epochs = 10000  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = min(2, len(df))  # Number of samples to train on.

In [6]:
input_texts = df['first_line'][:num_samples]
output_texts = df['title'][:num_samples]

In [7]:
input_texts = input_texts.apply(lambda i: i.strip().lower() + ' \n')
output_texts = output_texts.apply(lambda o: '\t '+ o.strip().lower() + ' \n')

In [8]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~', oov_token='<unk>')
tokenizer.fit_on_texts(input_texts)
tokenizer.fit_on_texts(output_texts)

In [9]:
input_sequences = tokenizer.texts_to_sequences(input_texts)
output_sequences = tokenizer.texts_to_sequences(output_texts)
word_id_dict = tokenizer.word_index
id_word_dict = dict()
for k in word_id_dict:
    id_word_dict[word_id_dict[k]] = k;

In [10]:
num_samples = len(input_texts)
max_encoder_seq_length = max([len(seq) for seq in input_sequences])
max_decoder_seq_length = max([len(seq) for seq in output_sequences])
num_dict_size = len(tokenizer.word_index)
# num_input_tokens = len(input_dict)
# num_output_tokens = len(output_dict)

In [11]:
print('Number of samples:', num_samples)
print('Max sequence length for inputs:', max_encoder_seq_length)
# print('Num of tokens for inputs:', num_input_tokens)
print('Max sequence length for outputs:', max_decoder_seq_length)
# print('Num of tokens for outputs:', num_output_tokens)
print('Number of words in the dictionary (including OOV token):', num_dict_size)

Number of samples: 2
Max sequence length for inputs: 42
Max sequence length for outputs: 11
Number of words in the dictionary (including OOV token): 73


In [12]:
# encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_input_tokens),dtype='float32')
# reversed_encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_input_tokens),dtype='float32')
# decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, num_output_tokens),dtype='float32')
# decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, num_output_tokens),dtype='float32')

In [13]:
# for i in range(num_samples):
#     input_text = input_tokenizer.texts_to_sequences(input_texts[i])
#     input_text = input_dict
#     encoder_input_data[i, :, :] = input_tokenizer.texts_to_sequences(input_texts[i])
#     reversed_encoder_input_data[i, :, :] = encoder_input_data[i, ::-1, :]
#     decoder_input_data[i, :, :] = output_tokenizer.texts_to_sequences(output_texts[i])
#     decoder_target_data[i, :, :] = np.row(decoder_input_data[i, :, :], 1)
#     decoder_target_data[i, 0, :] = 0;

In [14]:
## Baseline model: 1 LSTM encoder + 1 LSTM decoder
# Define an input sequence and process it.
encoder_inputs = Input(shape=(max_encoder_seq_length,))
# print(encoder_inputs.shape)
encoder_embedding = Embedding(num_dict_size, latent_dim)
encoder = encoder_embedding(encoder_inputs)
# print(encoder.shape)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_decoder_seq_length,))
decoder_embedding = Embedding(num_dict_size, latent_dim)
decoder = decoder_embedding(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder, _, _ = decoder_lstm(decoder, initial_state=encoder_states)
decoder = Flatten()(decoder)
decoder_dense = Dense(max_decoder_seq_length, activation='relu')
decoder_outputs = decoder_dense(decoder)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)# Compile & run training
model.compile(optimizer='adam', loss='mse')

model.summary()
# SVG(model_to_dot(model).create(prog='dot', format='svg'))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 42)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 42, 256)      18688       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 11, 256)      18688       input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [15]:
encoder_input_data = np.zeros((num_samples, max_encoder_seq_length), dtype='int32')
decoder_input_data = np.zeros((num_samples, max_decoder_seq_length), dtype='int32')
decoder_target_data = np.zeros((num_samples, max_decoder_seq_length), dtype='int32')

In [16]:
for k in range(num_samples):
    encoder_input_data[k][:len(input_sequences[k])] = input_sequences[k][:]
    decoder_input_data[k][:len(output_sequences[k])] = output_sequences[k][:]
    decoder_target_data[k][:len(output_sequences[k])-1] = output_sequences[k][1:]

In [17]:
# encoder_input_data = encoder_input_data.reshape((*encoder_input_data.shape, 1))
# decoder_input_data = decoder_input_data.reshape((*decoder_input_data.shape, 1))
# decoder_target_data = decoder_target_data.reshape((*decoder_target_data.shape, 1))

In [18]:
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs#,
          #validation_split=0.2
         )

Epoch 1/10000
2/2 [==============================] - 3s 1s/step - loss: 966.4792
Epoch 2/10000
2/2 [==============================] - 0s 54ms/step - loss: 966.0231
Epoch 3/10000
2/2 [==============================] - 0s 54ms/step - loss: 964.4863
Epoch 4/10000
2/2 [==============================] - 0s 55ms/step - loss: 962.2003
Epoch 5/10000
2/2 [==============================] - 0s 53ms/step - loss: 958.8226
Epoch 6/10000
2/2 [==============================] - 0s 42ms/step - loss: 953.3760
Epoch 7/10000
2/2 [==============================] - 0s 46ms/step - loss: 943.4501
Epoch 8/10000
2/2 [==============================] - 0s 48ms/step - loss: 921.7133
Epoch 9/10000
2/2 [==============================] - 0s 45ms/step - loss: 858.6315
Epoch 10/10000
2/2 [==============================] - 0s 55ms/step - loss: 718.5693
Epoch 11/10000
2/2 [==============================] - 0s 46ms/step - loss: 642.2038
Epoch 12/10000
2/2 [==============================] - 0s 45ms/step - loss: 596.7671
Epo

Epoch 99/10000
2/2 [==============================] - 0s 46ms/step - loss: 92.6249
Epoch 100/10000
2/2 [==============================] - 0s 61ms/step - loss: 86.6923
Epoch 101/10000
2/2 [==============================] - 0s 47ms/step - loss: 80.1432
Epoch 102/10000
2/2 [==============================] - 0s 48ms/step - loss: 74.2387
Epoch 103/10000
2/2 [==============================] - 0s 49ms/step - loss: 69.1123
Epoch 104/10000
2/2 [==============================] - 0s 55ms/step - loss: 64.2582
Epoch 105/10000
2/2 [==============================] - 0s 55ms/step - loss: 59.6224
Epoch 106/10000
2/2 [==============================] - 0s 46ms/step - loss: 55.0822
Epoch 107/10000
2/2 [==============================] - 0s 49ms/step - loss: 50.7098
Epoch 108/10000
2/2 [==============================] - 0s 58ms/step - loss: 46.5124
Epoch 109/10000
2/2 [==============================] - 0s 43ms/step - loss: 42.5043
Epoch 110/10000
2/2 [==============================] - 0s 53ms/step - loss: 3

Epoch 198/10000
2/2 [==============================] - 0s 54ms/step - loss: 0.4093
Epoch 199/10000
2/2 [==============================] - 0s 40ms/step - loss: 0.4092
Epoch 200/10000
2/2 [==============================] - 0s 47ms/step - loss: 0.4092
Epoch 201/10000
2/2 [==============================] - 0s 46ms/step - loss: 0.4092
Epoch 202/10000
2/2 [==============================] - 0s 47ms/step - loss: 0.4093
Epoch 203/10000
2/2 [==============================] - 0s 53ms/step - loss: 0.4093
Epoch 204/10000
2/2 [==============================] - 0s 48ms/step - loss: 0.4093
Epoch 205/10000
2/2 [==============================] - 0s 54ms/step - loss: 0.4092
Epoch 206/10000
2/2 [==============================] - 0s 38ms/step - loss: 0.4092
Epoch 207/10000
2/2 [==============================] - 0s 39ms/step - loss: 0.4092
Epoch 208/10000
2/2 [==============================] - 0s 51ms/step - loss: 0.4092
Epoch 209/10000
2/2 [==============================] - 0s 45ms/step - loss: 0.4092
Epoc

Epoch 297/10000
2/2 [==============================] - 0s 54ms/step - loss: 0.4091
Epoch 298/10000
2/2 [==============================] - 0s 56ms/step - loss: 0.4091
Epoch 299/10000
2/2 [==============================] - 0s 67ms/step - loss: 0.4091
Epoch 300/10000
2/2 [==============================] - 0s 55ms/step - loss: 0.4091
Epoch 301/10000
2/2 [==============================] - 0s 53ms/step - loss: 0.4091
Epoch 302/10000
2/2 [==============================] - 0s 50ms/step - loss: 0.4091
Epoch 303/10000
2/2 [==============================] - 0s 57ms/step - loss: 0.4091
Epoch 304/10000
2/2 [==============================] - 0s 57ms/step - loss: 0.4091
Epoch 305/10000
2/2 [==============================] - 0s 55ms/step - loss: 0.4091
Epoch 306/10000
2/2 [==============================] - 0s 56ms/step - loss: 0.4091
Epoch 307/10000
2/2 [==============================] - 0s 52ms/step - loss: 0.4091
Epoch 308/10000
2/2 [==============================] - 0s 48ms/step - loss: 0.4091
Epoc

Epoch 396/10000
2/2 [==============================] - 0s 48ms/step - loss: 0.4091
Epoch 397/10000
2/2 [==============================] - 0s 40ms/step - loss: 0.4091
Epoch 398/10000
2/2 [==============================] - 0s 55ms/step - loss: 0.4091
Epoch 399/10000
2/2 [==============================] - 0s 47ms/step - loss: 0.4091
Epoch 400/10000
2/2 [==============================] - 0s 47ms/step - loss: 0.4091
Epoch 401/10000
2/2 [==============================] - 0s 42ms/step - loss: 0.4091
Epoch 402/10000
2/2 [==============================] - 0s 52ms/step - loss: 0.4091
Epoch 403/10000
2/2 [==============================] - 0s 44ms/step - loss: 0.4091
Epoch 404/10000
2/2 [==============================] - 0s 38ms/step - loss: 0.4091
Epoch 405/10000
2/2 [==============================] - 0s 43ms/step - loss: 0.4091
Epoch 406/10000
2/2 [==============================] - 0s 60ms/step - loss: 0.4091
Epoch 407/10000
2/2 [==============================] - 0s 48ms/step - loss: 0.4091
Epoc

Epoch 495/10000
2/2 [==============================] - 0s 54ms/step - loss: 0.4091
Epoch 496/10000
2/2 [==============================] - 0s 40ms/step - loss: 0.4091
Epoch 497/10000
2/2 [==============================] - 0s 46ms/step - loss: 0.4091
Epoch 498/10000
2/2 [==============================] - 0s 48ms/step - loss: 0.4091
Epoch 499/10000
2/2 [==============================] - 0s 45ms/step - loss: 0.4091
Epoch 500/10000
2/2 [==============================] - 0s 47ms/step - loss: 0.4091
Epoch 501/10000
2/2 [==============================] - 0s 56ms/step - loss: 0.4091
Epoch 502/10000
2/2 [==============================] - 0s 48ms/step - loss: 0.4091
Epoch 503/10000
2/2 [==============================] - 0s 61ms/step - loss: 0.4091
Epoch 504/10000
2/2 [==============================] - 0s 62ms/step - loss: 0.4091
Epoch 505/10000
2/2 [==============================] - 0s 53ms/step - loss: 0.4091
Epoch 506/10000
2/2 [==============================] - 0s 55ms/step - loss: 0.4091
Epoc

Epoch 594/10000
2/2 [==============================] - 0s 54ms/step - loss: 0.4091
Epoch 595/10000
2/2 [==============================] - 0s 49ms/step - loss: 0.4091
Epoch 596/10000
2/2 [==============================] - 0s 55ms/step - loss: 0.4091
Epoch 597/10000
2/2 [==============================] - 0s 53ms/step - loss: 0.4091
Epoch 598/10000
2/2 [==============================] - 0s 62ms/step - loss: 0.4091
Epoch 599/10000
2/2 [==============================] - 0s 55ms/step - loss: 0.4091
Epoch 600/10000
2/2 [==============================] - 0s 54ms/step - loss: 0.4091
Epoch 601/10000
2/2 [==============================] - 0s 56ms/step - loss: 0.4091
Epoch 602/10000
2/2 [==============================] - 0s 54ms/step - loss: 0.4091
Epoch 603/10000
2/2 [==============================] - 0s 54ms/step - loss: 0.4091
Epoch 604/10000
2/2 [==============================] - 0s 56ms/step - loss: 0.4091
Epoch 605/10000
2/2 [==============================] - 0s 49ms/step - loss: 0.4091
Epoc

KeyboardInterrupt: 

In [19]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_mapped_input = decoder_embedding(decoder_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_mapped_input, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = Flatten()(decoder_outputs)
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
decoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 42)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 42, 256)           18688     
_________________________________________________________________
lstm_1 (LSTM)                [(None, 256), (None, 256) 525312    
Total params: 544,000
Trainable params: 544,000
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embe

In [20]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, max_decoder_seq_length), dtype='int32')
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = word_id_dict['\t']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    i = 1
    id_word_dict_cpy = id_word_dict.copy()
    id_word_dict_cpy[0] = '0'
    while not stop_condition:
#         print(target_seq)
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
#         print("output_token:", output_tokens)
#         sampled_char = id_word_dict[int(output_tokens)]
        decoded_sentence = str([id_word_dict_cpy[int(round(output_token))] for output_token in output_tokens[0]])
        if i > 14:
            stop_condition = True
        else:
            target_seq[0, 1:i] = output_tokens[0, 1:i]
            i += 1
#         # Exit condition: either hit max length
#         # or find stop character.
#         if (sampled_char == '\n' or
#            len(decoded_sentence) > max_decoder_seq_length):
#             stop_condition = True

#         # Update the target sequence (of length 1).
#         target_seq[0, i] = output_tokens
#         i += 1

#         # Update states
#         states_value = [h, c]

    return decoded_sentence

In [21]:
for seq_index in range(num_samples):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: indonesia’s finance minister said on monday the country will seek to host the headquarters for the asian infrastructure investment bank (aiib), a new multi-lateral lender that is led by china with beijing wanting the role for itself. 

Decoded sentence: ['sunday', 'new', '2', '\n', 'beijing', 'on', 'fire', 'for', 'aiib', '0', '0']
-
Input sentence: the san jose earthquakes became the latest team in major league soccer to move into their own purpose-built venue and they celebrated the opening game at the avaya stadium on sunday with a 2-1 win over the chicago fire. 

Decoded sentence: ['win', 'compete', 'earthquakes', 'stadium', '\n', 'win', '\t', 'over', '\n', '0', '0']
